In [1]:
import torch
import torch.nn as nn
import tiktoken

/workspaces/data-mining/.venv/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [2]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [ ]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

# feed forward is an implementation of the feed forward neural network
# it consists of two linear layers with a GELU activation function in between
# basically a mini neural network within the transformer architecture
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential( # it will execute the layers in sequence
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]), # goes from embedding dimension to 4 times embedding dimension, 768 --> 3072
            GELU(), # non linear activation function
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]), # goes back to embedding dimension 3072 --> 768
        )
        # We change the embedding dimension to 4 times its size to allow the model to learn more complex representations by having more parameters,
        # essentially massaging the input to extract information to be used in the next layer.

    def forward(self, x):
        return self.layers(x)

In [4]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method.

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [ ]:
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        self.drop_emb = nn.Dropout(cfg["drop_rate"])
        
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])
        
        self.final_norm = LayerNorm(cfg["emb_dim"])
        self.out_head = nn.Linear(
            cfg["emb_dim"], cfg["vocab_size"], bias=False
        )

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        tok_embeds = self.tok_emb(in_idx)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = tok_embeds + pos_embeds
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits


class TransformerBlock( nn.Module ):
    def __init__( self, cfg ):
        super().__init__()
        self.norm1 = LayerNorm( cfg["emb_dim"] )
        self.att = MultiHeadAttention( 
            d_in = cfg["emb_dim"], 
            d_out = cfg["emb_dim"], 
            context_length = cfg["context_length"], 
            dropout = cfg["drop_rate"], 
            num_heads = cfg["n_heads"] 
        )
        self.dropout = nn.Dropout( cfg["drop_rate"] )
        self.norm2 = LayerNorm( cfg["emb_dim"] )
        self.feedforward = FeedForward( cfg )
        
    def forward( self, x ):
        
        # the attention chunk in this transformer:
        shortcut = x 
        x = self.norm1( x )
        x = self.att( x )
        x = self.dropout( x )
        x = x + shortcut 
        
        # the feed forward chunk:
        shortcut = x
        x = self.norm2( x ) 
        x = self.feedforward( x )
        x = self.dropout( x )
        x = x + shortcut 
        
        return x 

# layer normalization helps improve training stability and efficiency of neural network training
# The main idea behind layer normalization is to adjust the activations (outputs)
# of a neural network layer to have a mean of 0 and a variance of 1, also
# known as unit variance. This adjustment speeds up the convergence to effective
# weights and ensures consistent, reliable training.


class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [6]:
tokenizer = tiktoken.get_encoding("gpt2")

In [7]:
batch = []

txt1 = "Every effort moves you"
txt2 = "Every day holds a"

batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)

tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])


In [8]:
model = GPTModel( GPT_CONFIG_124M )

logits = model(batch)
print("Output shape:", logits.shape)
print(logits)

Output shape: torch.Size([2, 4, 50257])
tensor([[[ 3.7263e-01,  1.1252e+00,  2.9818e-01,  ..., -4.8482e-01,
           3.1538e-01,  7.7558e-01],
         [-1.7536e+00,  8.2743e-01,  2.2460e-01,  ..., -5.4546e-01,
          -4.3120e-01, -5.4015e-01],
         [-2.0722e-01,  1.0607e-01,  6.6623e-01,  ...,  7.1515e-02,
           6.7146e-02,  4.1981e-01],
         [-3.7024e-01,  6.9460e-01, -4.0964e-01,  ...,  4.8787e-01,
          -9.3085e-04,  9.1818e-01]],

        [[-8.9386e-02,  7.6285e-01,  1.1915e-01,  ..., -4.0147e-01,
          -4.2705e-01,  6.7068e-01],
         [-1.3681e+00, -1.2766e-01,  1.0660e+00,  ..., -6.2290e-01,
          -9.2684e-01,  8.7634e-01],
         [ 1.3804e-01,  2.5953e-01, -5.7940e-01,  ...,  7.5097e-01,
          -6.6506e-01, -3.0861e-01],
         [-6.4766e-01,  4.1117e-01, -7.2484e-01,  ...,  1.3805e-01,
          -1.9508e-01, -3.8406e-01]]], grad_fn=<UnsafeViewBackward0>)


In [29]:
batch.numel()

8

In [32]:
total_params = sum(p.numel() for p in model.parameters() )
print(f"Total number of parameters: {total_params - model.out_head.weight.numel():,}")

Total number of parameters: 124,412,160


In [9]:
txt2

'Every day holds a'

In [10]:
logits.shape

torch.Size([2, 4, 50257])

In [ ]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    # idx is (batch, n_tokens) array of indices in the current context
    for _ in range(max_new_tokens):
        
        # Crop current context if it exceeds the supported context size
        # E.g., if LLM supports only 5 tokens, and the context size is 10
        # then only the last 5 tokens are used as context
        idx_cond = idx[:, -context_size:]
        
        # Get the predictions
        with torch.no_grad(): # supressing the generation of gradients graph, saves memory
            logits = model(idx_cond)
        
        # Focus only on the last time step
        # (batch, n_tokens, vocab_size) becomes (batch, vocab_size)
        logits = logits[:, -1, :]  # grab the last row of the logits

        # Apply softmax to get probabilities
        probas = torch.softmax(logits, dim=-1)  # (batch, vocab_size)

        # Get the idx of the vocab entry with the highest probability value
        idx_next = torch.argmax(probas, dim=-1, keepdim=True)  # (batch, 1), argmax looks up the index position with the highest value

        # Append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch, n_tokens+1)

    return idx

In [12]:
start_context = "Hello, I am"

encoded = tokenizer.encode(start_context)
print("encoded:", encoded)

encoded_tensor = torch.tensor(encoded).unsqueeze(0)
print("encoded_tensor.shape:", encoded_tensor.shape)

encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])


In [13]:
encoded

[15496, 11, 314, 716]

In [14]:
torch.tensor(encoded).unsqueeze(0).shape

torch.Size([1, 4])

In [15]:
out = generate_text_simple(
    model=model,
    idx=encoded_tensor, 
    max_new_tokens=6, 
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output:", out)
print("Output length:", len(out[0]))

Output: tensor([[15496,    11,   314,   716, 45920, 15859, 13184, 20014, 44279, 41856]])
Output length: 10


In [16]:
out.squeeze(0).tolist()

[15496, 11, 314, 716, 45920, 15859, 13184, 20014, 44279, 41856]

In [17]:
tokenizer.decode( out.squeeze(0).tolist() )

'Hello, I amigraph ACC inventory SummitstainedWHO'

In [18]:
def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

In [19]:
start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

In [20]:
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves youmma sidESH Oxford Suzо� criminal unablecor OpenGL


In [21]:
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,    1107, 588]])   #  "I really like"]

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
                        [1107,  588, 11311]]) #  " really like chocolate"]

In [22]:
with torch.no_grad():
    logits = model(inputs)

probas = torch.softmax(logits, dim=-1) # Probability of each token in vocabulary
print(probas.shape) # Shape: (batch_size, num_tokens, vocab_size)

torch.Size([2, 3, 50257])


In [23]:
probas 

tensor([[[2.0524e-05, 3.5077e-05, 1.7632e-05,  ..., 3.7440e-05,
          6.3079e-06, 2.0348e-05],
         [4.9434e-06, 3.9749e-05, 1.4736e-05,  ..., 1.4847e-05,
          8.8190e-06, 9.9155e-06],
         [1.5353e-05, 1.4868e-05, 2.6791e-05,  ..., 1.9223e-05,
          1.1563e-05, 1.7028e-05]],

        [[1.8475e-05, 2.6930e-05, 1.3223e-05,  ..., 3.6727e-05,
          7.5746e-06, 4.9635e-06],
         [1.2543e-05, 1.5647e-05, 2.9074e-05,  ..., 1.2159e-05,
          7.8718e-06, 7.7430e-06],
         [9.1236e-06, 2.5037e-05, 1.0622e-05,  ..., 1.7597e-05,
          5.8589e-06, 1.2524e-05]]])

In [24]:
# predicted tokens:
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print("Token IDs:\n", token_ids)

Token IDs:
 tensor([[[37408],
         [ 4121],
         [19613]],

        [[42032],
         [ 2506],
         [42861]]])


In [25]:
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")

Targets batch 1:  effort moves you
Outputs batch 1:  lushyan noon


In [26]:
text_idx = 0
target_probas_1 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 1:", target_probas_1)

text_idx = 1
target_probas_2 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 2:", target_probas_2)

Text 1: tensor([2.2100e-05, 1.8948e-05, 1.6939e-05])
Text 2: tensor([9.5776e-06, 1.0761e-05, 2.3484e-05])


In [27]:
# Compute logarithm of all token probabilities
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print(log_probas)

tensor([-10.7199, -10.8738, -10.9859, -11.5561, -11.4396, -10.6592])


In [28]:
avg_log_probas = torch.mean(log_probas)
print(avg_log_probas)
neg_avg_log_probas = avg_log_probas * -1
print(neg_avg_log_probas)

tensor(-11.0391)
tensor(11.0391)
